In [ ]:
import requests
import pandas as pd
import pandas_ta as ta
import seaborn as sns
import matplotlib.pyplot as plt
import time
import os


class portfolio_insider():
    def __init__(self, atr_length=50):
        self.api_key = 'YOUR-API-KEY' #https://www.alphavantage.co/
        self.filename = input('Stock-List Name? ')
        
        self.returns_df = pd.DataFrame()
        self.atr_sma_df = pd.DataFrame()

        with open(self.filename, 'r') as file:
            self.stocks = [line.strip() for line in file.readlines()]

        print(f'This may take up to {len(self.stocks) * 12 + 3}s.\n')

        for symbol in self.stocks:
            print(f"Requesting data for {symbol}...")
            url = (
                f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY'
                f'&symbol={symbol}&outputsize=full&apikey={self.api_key}'
            )

            response = requests.get(url)
            data = response.json()

            if "Note" in data:
                print(f"⚠️ API limit reached: {data['Note']}")
                return

            if "Time Series (Daily)" not in data:
                print(f"❌ Error at {symbol}: {data}")
                continue

            ts_data = data["Time Series (Daily)"]
            df = pd.DataFrame.from_dict(ts_data, orient='index')
            df.index = pd.to_datetime(df.index)
            df = df.rename(columns={
                '1. open': 'Open',
                '2. high': 'High',
                '3. low': 'Low',
                '4. close': 'Close'
            })

            try:
                df = df[['Open', 'High', 'Low', 'Close']].astype(float).sort_index()
            except Exception as e:
                print(f"⚠️ Error converting {symbol} data: {e}")
                continue

            returns = df['Close'].pct_change().dropna()
            if self.returns_df.empty:
                self.returns_df = returns.to_frame(name=symbol)
            else:
                self.returns_df = self.returns_df.join(returns.to_frame(name=symbol), how='inner')

            atr = ta.atr(high=df['High'], low=df['Low'], close=df['Close'], length=atr_length)
            atr_sma = atr.rolling(window=atr_length).mean()

            self.atr_sma_df[symbol] = atr_sma

            time.sleep(12)

    def get_correlations(self):
        if self.returns_df.empty:
            print("❌ No return data available.")
            return

        corr = self.returns_df.corr()
        sns.heatmap(corr, cbar=True, cmap='coolwarm', annot=True)
        plt.title("Correlation Matrix")
        plt.tight_layout()
        plt.show()

    def get_atr(self):
        try:
            atr_length = int(input('ATR smoothing length (e.g. 14)? '))
            if atr_length < 1:
                raise ValueError
        except ValueError:
            print("Invalid input. Must be an integer >= 1.")
            return

        if self.atr_sma_df.empty:
            print("❌ No ATR data available.")
            return

        for symbol in self.atr_sma_df.columns:
            plt.plot(self.atr_sma_df[symbol], label=symbol, linewidth=0.8, alpha=0.7)

        mean_atr = self.atr_sma_df.mean(axis=1)
        plt.plot(mean_atr, label='Mean ATR', linewidth=1.5, color='red')

        plt.title(f'Smoothed ATR ({atr_length}d)')
        plt.xlabel('Date')
        plt.ylabel('ATR')
        plt.legend()
        plt.grid(True, alpha=0.2)
        plt.tight_layout()
        plt.show()
        
client_1 = portfolio_insider()
client_1.get_correlations()
client_1.get_atr()      